## Aggregated table loading into gold incrementally

Reading data from Silver using PySpark

In [0]:
df_fact_transactions = spark.read.format("delta").load("abfss://silver@dlgysnergy.dfs.core.windows.net/stg_hier_fact_transactions_sl")
df_fact_transactions.createOrReplaceTempView("stg_fact_transactions")


In [0]:
%sql
SELECT * FROM stg_fact_transactions
LIMIT 10;


Aggregating the weekly sales using SQL

In [0]:
%sql
CREATE OR REPLACE TEMP VIEW agg_weekly_sales AS
SELECT
  pos_site_id,
  sku_id,
  fscldt_id,
  price_substate_id,
  type,
  SUM(sales_units) AS total_sales_units,
  SUM(sales_dollars) AS total_sales_dollars,
  SUM(discount_dollars) AS total_discount_dollars
FROM stg_fact_transactions
GROUP BY pos_site_id, sku_id, fscldt_id, price_substate_id, type;


In [0]:
%sql
SELECT * FROM agg_weekly_sales
LIMIT 10;


Merge changes into Gold layer using CDC logic(incremental load)

In [0]:
%sql
CREATE TABLE IF NOT EXISTS delta.`abfss://gold@dlgysnergy.dfs.core.windows.net/mview_weekly_sales`
AS
SELECT * FROM agg_weekly_sales

In [0]:
%sql
MERGE INTO delta.`abfss://gold@dlgysnergy.dfs.core.windows.net/mview_weekly_sales` AS target
USING agg_weekly_sales AS source
ON  target.pos_site_id = source.pos_site_id
AND target.sku_id = source.sku_id
AND target.fscldt_id = source.fscldt_id
AND target.price_substate_id = source.price_substate_id
AND target.type = source.type
WHEN MATCHED THEN UPDATE SET
  target.total_sales_units = source.total_sales_units,
  target.total_sales_dollars = source.total_sales_dollars,
  target.total_discount_dollars = source.total_discount_dollars
WHEN NOT MATCHED THEN INSERT (
  pos_site_id,
  sku_id,
  fscldt_id,
  price_substate_id,
  type,
  total_sales_units,
  total_sales_dollars,
  total_discount_dollars
) VALUES (
  source.pos_site_id,
  source.sku_id,
  source.fscldt_id,
  source.price_substate_id,
  source.type,
  source.total_sales_units,
  source.total_sales_dollars,
  source.total_discount_dollars
);


In [0]:

# Optional: Verify data in Gold Layer
df_gold = spark.read.format("delta").load("abfss://gold@dlgysnergy.dfs.core.windows.net/mview_weekly_sales")
df_gold.limit(20).display()
